In [1]:
from vnpy.trader.constant import Exchange,Interval
from data_analysis import DataAnalysis
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
datetime.now()

datetime.datetime(2021, 6, 14, 22, 41, 54, 647397)

In [3]:
herramiento = DataAnalysis()
herramiento.load_history(    
    symbol="ethusdt", 
    exchange=Exchange.BINANCE, 
    interval=Interval.MINUTE, 
    start=datetime(2021, 6, 10), 
    end=datetime.now(),
    rate = 8/10000,
    index_3to1 = ["ATR","CCI"],
    index_1to1 = ["STDDEV","SMA"],
    index_2to2 = ["AROON"],
    index_2to1 = ["AROONOSC"],
    index_4to1 = ["BOP"],
    window_index=30,
)

2021-06-14 22:41:54.681330	开始加载历史数据
2021-06-14 22:41:56.135839	历史数据加载完成，数据量：7122


In [4]:
herramiento.orignal

,open,high,low,close,volume
2021-06-10 00:00:00+08:00,2566.94,2579.93,2566.93,2575.15,4104.62054
2021-06-10 00:01:00+08:00,2575.26,2590.00,2570.05,2574.30,4568.77672
2021-06-10 00:02:00+08:00,2574.30,2581.43,2567.83,2577.59,2813.00977
2021-06-10 00:03:00+08:00,2577.37,2578.69,2571.00,2576.53,1302.64105
2021-06-10 00:04:00+08:00,2576.20,2586.98,2575.47,2580.64,1844.81815
...,...,...,...,...,...
2021-06-14 22:37:00+08:00,2569.26,2569.49,2565.21,2568.89,1125.62269
2021-06-14 22:38:00+08:00,2568.89,2570.78,2568.89,2570.40,284.04619
2021-06-14 22:39:00+08:00,2570.49,2570.63,2568.53,2569.76,357.73234
2021-06-14 22:40:00+08:00,2569.75,2570.55,2568.71,2569.10,294.11582


In [5]:
df = herramiento.orignal
df.head()

,open,high,low,close,volume
2021-06-10 00:00:00+08:00,2566.94,2579.93,2566.93,2575.15,4104.62054
2021-06-10 00:01:00+08:00,2575.26,2590.00,2570.05,2574.30,4568.77672
2021-06-10 00:02:00+08:00,2574.30,2581.43,2567.83,2577.59,2813.00977
2021-06-10 00:03:00+08:00,2577.37,2578.69,2571.00,2576.53,1302.64105
2021-06-10 00:04:00+08:00,2576.20,2586.98,2575.47,2580.64,1844.81815


In [6]:
df.tail()

,open,high,low,close,volume
2021-06-14 22:37:00+08:00,2569.26,2569.49,2565.21,2568.89,1125.62269
2021-06-14 22:38:00+08:00,2568.89,2570.78,2568.89,2570.40,284.04619
2021-06-14 22:39:00+08:00,2570.49,2570.63,2568.53,2569.76,357.73234
2021-06-14 22:40:00+08:00,2569.75,2570.55,2568.71,2569.10,294.11582
2021-06-14 22:41:00+08:00,2569.10,2569.47,2567.29,2567.44,300.94437


#### 合成半小时K线数据
data30 = pd.DataFrame()
interval='30T'
data30["open"] = df["open"].resample(interval).first()
data30["high"] = df["high"].resample(interval).max()
data30["low"] = df["low"].resample(interval).min()
data30["close"] = df["close"].resample(interval).last()
data30["volume"] = df["volume"].resample(interval).sum()

参数配置

In [7]:
#标准差
STD = 0.25

#滑动窗口小时数
rolling_hours_window = 2

# 套利摩擦
arb_fri = 0.001

# 千三池 0.006， 万五池 0.001
bar_len = 0.001



In [8]:
len_data = len(df["close"])

In [9]:
len_data

7122

In [10]:
df.iloc[0]

open      2566.94000
high      2579.93000
low       2566.93000
close     2575.15000
volume    4104.62054
Name: 2021-06-10 00:00:00+08:00, dtype: float64

In [11]:
df["pre_close"] = df["close"].shift(1).fillna(0)

In [12]:
df['log_return'] = np.log(df['close']/df['pre_close'])

In [13]:
df['log_return'].rolling(30).std()

2021-06-10 00:00:00+08:00         NaN
2021-06-10 00:01:00+08:00         NaN
2021-06-10 00:02:00+08:00         NaN
2021-06-10 00:03:00+08:00         NaN
2021-06-10 00:04:00+08:00         NaN
                               ...   
2021-06-14 22:37:00+08:00    0.000947
2021-06-14 22:38:00+08:00    0.000934
2021-06-14 22:39:00+08:00    0.000932
2021-06-14 22:40:00+08:00    0.000933
2021-06-14 22:41:00+08:00    0.000939
Name: log_return, Length: 7122, dtype: float64

In [14]:
df['rolling_sigma'] = df['log_return'].rolling(rolling_hours_window*60).std()
df['daily_sigma'] = df['rolling_sigma']*np.sqrt(1440)

In [15]:
df.iloc[119:125]

,open,high,low,close,volume,pre_close,log_return,rolling_sigma,daily_sigma
2021-06-10 01:59:00+08:00,2587.73,2591.82,2587.71,2589.22,964.08574,2587.74,0.000572,NaN,NaN
2021-06-10 02:00:00+08:00,2589.22,2595.00,2589.22,2594.24,1453.25232,2589.22,0.001937,0.001715,0.065084
2021-06-10 02:01:00+08:00,2594.50,2614.00,2594.50,2604.65,2973.13094,2594.24,0.004005,0.001752,0.066486
2021-06-10 02:02:00+08:00,2604.50,2605.21,2598.67,2599.48,1463.72522,2604.65,-0.001987,0.001759,0.066746
2021-06-10 02:03:00+08:00,2599.48,2599.48,2592.60,2596.00,1058.86537,2599.48,-0.001340,0.001763,0.066904
2021-06-10 02:04:00+08:00,2596.00,2599.96,2588.93,2588.96,702.12309,2596.00,-0.002716,0.001775,0.067375


In [16]:
df['up_float'] = np.exp(df['daily_sigma']*STD)
df['down_float'] = np.exp(-df['daily_sigma']*STD)

In [17]:
df['target_range'] = np.round(np.log(df['up_float'])/np.log(1+bar_len),0)

In [18]:
df['target_range'].max()

18.0

In [19]:
df['target_range'].min()

6.0

In [20]:
df.index = df.index.tz_localize(None)

In [21]:
df.index

DatetimeIndex(['2021-06-10 00:00:00', '2021-06-10 00:01:00',
               '2021-06-10 00:02:00', '2021-06-10 00:03:00',
               '2021-06-10 00:04:00', '2021-06-10 00:05:00',
               '2021-06-10 00:06:00', '2021-06-10 00:07:00',
               '2021-06-10 00:08:00', '2021-06-10 00:09:00',
               ...
               '2021-06-14 22:32:00', '2021-06-14 22:33:00',
               '2021-06-14 22:34:00', '2021-06-14 22:35:00',
               '2021-06-14 22:36:00', '2021-06-14 22:37:00',
               '2021-06-14 22:38:00', '2021-06-14 22:39:00',
               '2021-06-14 22:40:00', '2021-06-14 22:41:00'],
              dtype='datetime64[ns]', length=7122, freq=None)

In [22]:
df.index[10:12]

DatetimeIndex(['2021-06-10 00:10:00', '2021-06-10 00:11:00'], dtype='datetime64[ns]', freq=None)

In [23]:
#df.to_excel('test.xlsx')

In [24]:
df['close_rolling_30_max'] = df['close'].rolling(30).max()
df['close_rolling_30_min'] = df['close'].rolling(30).min()

In [25]:
# 初始化
start_index = '2021-06-14 10:38:00'
df.loc[start_index,'adjust_signal'] = 1
df.loc[start_index,'actural_range'] = df.loc[start_index,'target_range']
df.loc[start_index,'benchmark_price'] = df.loc[start_index]['pre_close']
df.loc[start_index,'a'] = df.loc[start_index]['benchmark_price']*df.loc[start_index]['down_float']*(1-arb_fri)
df.loc[start_index,'b'] = df.loc[start_index]['benchmark_price']*df.loc[start_index]['up_float']*(1+arb_fri)
a  = df.loc[start_index]['benchmark_price']*df.loc[start_index]['down_float']*(1-arb_fri)
b = df.loc[start_index]['benchmark_price']*df.loc[start_index]['up_float']*(1+arb_fri)

df_1 = df[start_index:]
#df_1['close_rolling_30_max'] = df_1['close'].rolling(30).max()
#df_1['close_rolling_30_min'] = df_1['close'].rolling(30).min()
data_len = len(df_1)

start_index_list=[]

In [26]:
df_1

,open,high,low,close,volume,pre_close,log_return,rolling_sigma,daily_sigma,up_float,down_float,target_range,close_rolling_30_max,close_rolling_30_min,adjust_signal,actural_range,benchmark_price,a,b
2021-06-14 10:38:00,2498.12,2498.12,2495.23,2495.49,125.67443,2498.43,-0.001177,0.001216,0.046153,1.011605,0.988528,12.0,2509.00,2493.15,1.0,12.0,2498.43,2467.298586,2529.951688
2021-06-14 10:39:00,2495.45,2495.80,2493.21,2493.21,280.28675,2495.49,-0.000914,0.001211,0.045940,1.011551,0.988581,11.0,2509.00,2493.15,NaN,NaN,NaN,NaN,NaN
2021-06-14 10:40:00,2493.21,2494.00,2492.39,2492.50,141.31638,2493.21,-0.000285,0.001200,0.045539,1.011450,0.988680,11.0,2509.00,2492.50,NaN,NaN,NaN,NaN,NaN
2021-06-14 10:41:00,2492.50,2495.15,2492.45,2495.15,322.99272,2492.50,0.001063,0.001202,0.045598,1.011465,0.988665,11.0,2509.00,2492.50,NaN,NaN,NaN,NaN,NaN
2021-06-14 10:42:00,2495.14,2496.62,2494.31,2494.68,324.43642,2495.15,-0.000188,0.001202,0.045600,1.011465,0.988665,11.0,2508.96,2492.50,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-14 22:37:00,2569.26,2569.49,2565.21,2568.89,1125.62269,2569.26,-0.000144,0.001388,0.052679,1.013257,0.986917,13.0,2575.89,2555.81,NaN,NaN,NaN,NaN,NaN
2021-06-14 22:38:00,2568.89,2570.78,2568.89,2570.40,284.04619,2568.89,0.000588,0.001388,0.052669,1.013254,0.986919,13.0,2575.89,2555.81,NaN,NaN,NaN,NaN,NaN
2021-06-14 22:39:00,2570.49,2570.63,2568.53,2569.76,357.73234,2570.40,-0.000249,0.001388,0.052655,1.013251,0.986923,13.0,2575.89,2555.81,NaN,NaN,NaN,NaN,NaN
2021-06-14 22:40:00,2569.75,2570.55,2568.71,2569.10,294.11582,2569.76,-0.000257,0.001388,0.052684,1.013258,0.986915,13.0,2575.89,2555.81,NaN,NaN,NaN,NaN,NaN


In [27]:
for i in range(1,data_len):
    if (df_1['close_rolling_30_max'][i]<a) | (df_1['close_rolling_30_min'][i]>b):
        df_1['adjust_signal'][i] = 1
        df_1['benchmark_price'][i] = df_1['pre_close'][i]
        
        df_1['a'][i] = df_1['benchmark_price'][i]*df_1['down_float'][i]*(1-arb_fri)
        df_1['b'][i] = df_1['benchmark_price'][i]*df_1['up_float'][i]*(1+arb_fri)
        a = df_1['benchmark_price'][i]*df_1['down_float'][i]*(1-arb_fri)
        b = df_1['benchmark_price'][i]*df_1['up_float'][i]*(1+arb_fri)
        
        df_1['actural_range'][i] = np.round(np.log(df_1['up_float'][i])/np.log(1+bar_len),0)
        start_index = df_1.index[i]
        start_index_list.append(start_index)
        #break
    else:
        df_1['adjust_signal'][i] = 0
        df_1['benchmark_price'][i] = df_1['benchmark_price'][start_index]
        df_1['a'][i] =  df_1['a'][start_index]
        df_1['b'][i] =  df_1['b'][start_index]
        df_1['actural_range'][i] =  df_1['target_range'][start_index]
        continue

In [28]:
start_index_list

[Timestamp('2021-06-14 21:51:00')]

In [29]:
df_1.tail()

,open,high,low,close,volume,pre_close,log_return,rolling_sigma,daily_sigma,up_float,down_float,target_range,close_rolling_30_max,close_rolling_30_min,adjust_signal,actural_range,benchmark_price,a,b
2021-06-14 22:37:00,2569.26,2569.49,2565.21,2568.89,1125.62269,2569.26,-0.000144,0.001388,0.052679,1.013257,0.986917,13.0,2575.89,2555.81,0.0,13.0,2560.54,2525.870505,2595.682764
2021-06-14 22:38:00,2568.89,2570.78,2568.89,2570.40,284.04619,2568.89,0.000588,0.001388,0.052669,1.013254,0.986919,13.0,2575.89,2555.81,0.0,13.0,2560.54,2525.870505,2595.682764
2021-06-14 22:39:00,2570.49,2570.63,2568.53,2569.76,357.73234,2570.40,-0.000249,0.001388,0.052655,1.013251,0.986923,13.0,2575.89,2555.81,0.0,13.0,2560.54,2525.870505,2595.682764
2021-06-14 22:40:00,2569.75,2570.55,2568.71,2569.10,294.11582,2569.76,-0.000257,0.001388,0.052684,1.013258,0.986915,13.0,2575.89,2555.81,0.0,13.0,2560.54,2525.870505,2595.682764
2021-06-14 22:41:00,2569.10,2569.47,2567.29,2567.44,300.94437,2569.10,-0.000646,0.001390,0.052745,1.013274,0.986900,13.0,2575.89,2555.81,0.0,13.0,2560.54,2525.870505,2595.682764


In [30]:
df_1['adjust_signal'].sum()

2.0

In [31]:
#df_1.to_excel('test3.xlsx')